# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import re
import nltk

#nltk.download(['punkt', 'wordnet','stopwords','omw-1.4'])

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import pandas as pd
import numpy as np 

from sqlalchemy import create_engine

#--------------------------------------------------------

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

#--------------------------------------------------------

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer

In [2]:
# load data from database
engine = create_engine('sqlite:///./data/DisasterResponse.db')
df = pd.read_sql_table('message', engine)

# X : the Features , Y : the catogeries
Y = df.iloc[:,4:].values
X = df['message'].values

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/14/e3q8)

### 2. Write a tokenization function to process your text data

In [3]:
# define stop words
stop = stopwords.words("english")
# append some more of the stopword
stop.append('us')
stop.append('pass')
stop.append('could')
stop.append('may')
stop.append('maybe')
# remove these stop word becasue it affects the model positively
stop.remove('in')
stop.remove('here')  

def tokenize(text):
    '''
    
    applying NLP process on the text
    
    inputs:
    
    text: String
       
    outputs:

    list of cleaned text
    
    '''

    text = text.lower()

    # nothing but the alphapets
    text = re.sub(r"[^a-z]+"," ",text)
    
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stop]

    clean_tokens = []

    # for loop to stem the word
    
    lemmatizer = WordNetLemmatizer()
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        if tok == clean_tok :
           clean_tok = lemmatizer.lemmatize(tok,'v').strip()
        if tok == clean_tok :
           clean_tok = lemmatizer.lemmatize(tok,'a').strip()
        if tok == clean_tok :
           clean_tok = lemmatizer.lemmatize(tok,'r').strip()
        if tok == clean_tok :
           clean_tok = lemmatizer.lemmatize(tok,'s').strip()
        clean_tokens.append(clean_tok)

    # delete every word that is less than 2 alphabet except in and at    
    clean_tokens = [w for w in clean_tokens if (len(w) > 2) | (w in ['in','at'])]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline(
        [
        ('vect', CountVectorizer(tokenizer=tokenize,min_df= 0.002)),
        ('tfidf', TfidfTransformer()),  
        ('MltRFC', MultiOutputClassifier(RandomForestClassifier()))
        ]
        ) 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
# split the dataset to Test and Train DATA
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=.15 ,
                                   random_state=8)
# fit the pipeine
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(min_df=0.002,
                                 tokenizer=<function tokenize at 0x00000201E0441240>)),
                ('tfidf', TfidfTransformer()),
                ('MltRFC',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred = pipeline.predict(X_test)

category_names = list(df.iloc[:,4:].columns)

for i in range(len(category_names)):
        print(category_names[i],'\n', classification_report(y_test[:,i], 
        y_pred[:,i]))

# accuracy of the model
accuracy = (y_pred == y_test).mean()
print("Accuracy : -\n", round(accuracy.mean()*100,4) ,"%")

related 
               precision    recall  f1-score   support

           1       1.00      1.00      1.00      3014

    accuracy                           1.00      3014
   macro avg       1.00      1.00      1.00      3014
weighted avg       1.00      1.00      1.00      3014

request 
               precision    recall  f1-score   support

           0       0.88      0.96      0.92      2309
           1       0.81      0.59      0.68       705

    accuracy                           0.87      3014
   macro avg       0.85      0.77      0.80      3014
weighted avg       0.87      0.87      0.86      3014

offer 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      2993
           1       0.00      0.00      0.00        21

    accuracy                           0.99      3014
   macro avg       0.50      0.50      0.50      3014
weighted avg       0.99      0.99      0.99      3014

aid_related 
               precision    reca

d:\python\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\python\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\python\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


other_aid 
               precision    recall  f1-score   support

           0       0.84      0.99      0.91      2500
           1       0.61      0.08      0.15       514

    accuracy                           0.83      3014
   macro avg       0.73      0.54      0.53      3014
weighted avg       0.80      0.83      0.78      3014

infrastructure_related 
               precision    recall  f1-score   support

           0       0.91      1.00      0.96      2758
           1       0.00      0.00      0.00       256

    accuracy                           0.91      3014
   macro avg       0.46      0.50      0.48      3014
weighted avg       0.84      0.91      0.87      3014

transport 
               precision    recall  f1-score   support

           0       0.95      0.99      0.97      2836
           1       0.67      0.17      0.27       178

    accuracy                           0.95      3014
   macro avg       0.81      0.58      0.62      3014
weighted avg       0.93  

d:\python\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\python\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\python\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\python\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precisi

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
parameters = {
   'vect__min_df': [0,0.001,0.002,0.005],
   'tfidf__smooth_idf': [False , True],
   'tfidf__use_idf': [False , True]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
cv.fit(X_train, y_train)

y_pred = cv.predict(X_test)

# accuracy of the best estimator carrying the best cobination of parameters 
accuracy = (y_pred == y_test).mean()
print("Accuracy :-\n", round(accuracy.mean()*100,4) ,"%")

Fitting 2 folds for each of 16 candidates, totalling 32 fits
[CV 1/2] END tfidf__smooth_idf=False, tfidf__use_idf=False, vect__min_df=0;, score=0.231 total time= 7.5min
[CV 2/2] END tfidf__smooth_idf=False, tfidf__use_idf=False, vect__min_df=0;, score=0.225 total time= 8.8min
[CV 1/2] END tfidf__smooth_idf=False, tfidf__use_idf=False, vect__min_df=0.001;, score=0.242 total time= 3.6min
[CV 2/2] END tfidf__smooth_idf=False, tfidf__use_idf=False, vect__min_df=0.001;, score=0.243 total time= 3.6min
[CV 1/2] END tfidf__smooth_idf=False, tfidf__use_idf=False, vect__min_df=0.002;, score=0.246 total time= 2.9min
[CV 2/2] END tfidf__smooth_idf=False, tfidf__use_idf=False, vect__min_df=0.002;, score=0.237 total time= 3.1min
[CV 1/2] END tfidf__smooth_idf=False, tfidf__use_idf=False, vect__min_df=0.005;, score=0.244 total time= 2.4min
[CV 2/2] END tfidf__smooth_idf=False, tfidf__use_idf=False, vect__min_df=0.005;, score=0.236 total time= 2.4min
[CV 1/2] END tfidf__smooth_idf=False, tfidf__use_id

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# regex to support the X_transform function
url_re = '[a-zA-Z]*http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+[a-zA-Z]*'
email_re = '[a-zA-Z_0-9-+]+@(.?[a-zA-Z_0-9-]+\.[a-zA-Z_0-9-]+)+$'
url = re.compile(url_re)
eml = re.compile(email_re)

def X_Transform(X):
    '''
    
    transform the feature column to 4 features columns
    
    inputs:
    
    X : one column of the message
       
    outputs:

    2d array of containing and 4 columns of :-

    message : message text without numbers , emails and url
    number : 0 or 1 if the text has a number
    email : 0 or 1 if the text has an email
    url : 0 or 1 if the text has an url

    
    '''
    Array = np.zeros((len(X),4),dtype=object)
    
    for i,x in enumerate(X) :
        Array[i,0] = x
        if url.search(x):
            Array[i,3] = 1   
            text = url.sub(" ",x)
            Array[i,0] = text    
        if eml.search(x):
            Array[i,2] = 1   
            text = eml.sub(" ",x)
            Array[i,0] = text        
        if re.search(r'[0-9]+',x):
            Array[i,1] = 1   
            text = re.sub(r'[a-zA-Z]*[0-9]+[a-zA-Z]*'," ",x)
            Array[i,0] = text     
            
    return Array      

In [ ]:
pipeline = Pipeline(
        [
        ('vect', CountVectorizer(tokenizer=tokenize,min_df=0.002)),
        ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True))
        ]
        ) 

ct = ColumnTransformer( [('CatTr',pipeline,0)] , remainder='passthrough') 

Newpi = Pipeline([ ('cr', ct) ,  ('MltRFC',
                MultiOutputClassifier(RandomForestClassifier())) ])

Newpi.fit(X_train, y_train)

y_pred = Newpi.predict(X_test)

# accuracy of the improved model
accuracy = (y_pred == y_test).mean()
print("Accuracy :-\n", round(accuracy.mean()*100,4) ,"%")

### 9. Export your model as a pickle file

In [ ]:
import pickle
filename = 'classifier.pkl'
pickle.dump(Newpi, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.